In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('ml-diabetes').getOrCreate

In [ ]:
df=spark.read.csv('/content/diabetes (1).csv',header=True,inferSchema=True)
df.printSchema()

AttributeError: 'function' object has no attribute 'read'

In [ ]:
import pandas as pd
pd.DataFrame(df.take(5),columns=df.columns).transpose

In [ ]:
df.show()

In [ ]:
df.toPandas()

In [ ]:
df.groupby('Outcome').count().toPandas()

In [ ]:
# making numerical features
numeric_features=[t[0] for t in df.types if t[1]=='int']

In [ ]:
numeric_features

In [ ]:
df.select(numeric_features).describe().toPandas().transpose()

In [ ]:
from pandas.plotting import scatter_matrix
numeric_data=df.select(numeric_features).toPandas()

axs=scatter_matrix(numeric_data,figsize=(8,8));

# Rotate axis labels and remove axis ticks
n=len(numeric_data.columns)
for i in range(n):
  v=axis[i,0]
  v.yaxis.label.set_rotation(0)
  v.yaxis.label.set_ha('right')
  v.set_yticks(())
  h=axs[n-1,i]
  h.xaxis.label.set_rotation(90)
  h.set_xticks(())

In [ ]:
from pyspark.sql.functions import isnull,when,count,col

In [ ]:
df.select([count(when(isnull(c),c)).alias(c) for c in df.columns]).show()

In [ ]:
dataset=df.drop('SkinThickness')
dataset=dataset.drop('Insulin')
dataset_new=dataset.drop('DiabetesPedigreeFunction')
dataset_final=dataset_new.drop('Pregnancies')

dataset_final.show()

In [ ]:
required_features=['Glucose','BloodPressure','BMI','Age']
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=required_features,outputCol='features')

treansformed_data=assembler.transform(dataset_final)
transformed_data.show()

In [ ]:
# Split the data
(training_data,test_data)=transformed_data.randomSplit([0.8,0.2],seed=10)

print('Training Dataset Count:'+str(training_data.count()))
print('Test Dataset Count:'+ str(test_data.count()))

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr=LogisticRegression(featuresCol='features',labelCol='Outcome',maxIter=50)
lrModel=lr.fit(training_data)
lr_predictions=lrModel.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'Outcome', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(lr_predictions))